In [4]:
from crapstack.document_stores.faiss import FAISSDocumentStore


In [5]:
document_store = FAISSDocumentStore(embedding_dim=128, faiss_index_factory_str="Flat")


In [6]:
import pandas as pd

data = pd.read_csv('c:/users/kchun/desktop/notebooks/patents_v6.csv')
data

,patent_id,mainclass_id,year,patent_text
0,3930300,257,1976,Junction field effect transistor A junction f...
1,3930303,257,1976,Method manufacturing compact thermoelectric mo...
2,3930304,257,1976,Method apparatus selective burnout trimming in...
3,3930305,257,1976,Method manufacturing integrated circuits A met...
4,3930306,257,1976,Process attaching lead member semiconductor de...
...,...,...,...,...
7814692,8496534,470,2013,Group taps prepared hole cutting tools In FIG ...
7814693,8534022,470,2013,Twisted threaded reinforcing bar Techniques re...
7814694,8550755,470,2013,Tap driver rigidsynchronous tapping Disclosed ...
7814695,8602696,470,2013,Form tap plurality lobes A form tap tapping ar...


In [7]:
from crapstack.haystack.schema import Document

documents = [Document(content=txt) for txt in data['patent_text'].sample(frac=.0005)]
document_store.write_documents(documents=documents)

Writing Documents: 10000it [00:03, 2670.84it/s]            


In [8]:
from crapstack.haystack.nodes.retriever import DensePassageRetriever
from crapstack.haystack.nodes.answer_generator.transformers import Seq2SeqGenerator
from crapstack.haystack.pipelines.standard_pipelines import GenerativeQAPipeline
# from transformers import T5ForConditionalGeneration
# import torch

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="vblagoje/dpr-question_encoder-single-lfqa-wiki",
    passage_embedding_model="vblagoje/dpr-ctx_encoder-single-lfqa-wiki",
)

generator = Seq2SeqGenerator(model_name_or_path="vblagoje/bart_lfqa")
# generator = T5ForConditionalGeneration.from_pretrained("google/flan-ul2", torch_dtype=torch.bfloat16, device_map="auto")                                                                 

pipe = GenerativeQAPipeline(generator, retriever)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
ML logging didn't work: name 'tracker' is not defined


In [9]:
document_store.update_embeddings(retriever)

Documents Processed: 10000 docs [10:44, 15.52 docs/s]           


In [10]:
prediction = pipe.run(
    query="what is machine learning?"
)

prediction

{'query': 'what is machine learning?',
 'answers': [<Answer {'answer': "Machine learning is the process of learning to recognize patterns in data. For example, if you give a computer a picture of a dog, it will learn to recognize the shape of the dog's face. Machine learning can be used to learn to recognise patterns in other data as well.", 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': ['c27df76a32b18dbfa05ea6ebc2b8fe3d', 'af48f971606dcd26008ba961ecf292de', '47a6ce513b6ab680ce9263b6677363df', '51bbd03dcf12deb918906d18cdd9ae9c', '88ffdd64bab0e31f7a2532544024608b', 'e3b314abe89d00e93fd61e1f978ebdef', '1f5d55f3aec8a3f5b9911b4e073a457c', 'c461e5fe3b4b142ff75c257d472425b2', 'bfa158f4d433bccdb06c781753c43275', '487490e188415989db94731927bbdcae'], 'meta': {'doc_scores': [0.5382860120504362, 0.5381226770061404, 0.5369550752082091, 0.5355375157624958, 0.5327604277313529, 0.5311598659687891, 0.5308309394487241, 0.5